In [1]:
import pandas as pd
import numpy as np
import random
import tensorflow as tf
print(tf.__version__)

2.7.0


In [2]:
SEED=12
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)
print("SEED 고정 :", SEED)

SEED 고정 : 12


In [3]:
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')
submission = pd.read_csv('./data/sample_submission.csv')

print(train.shape, test.shape, submission.shape)

(1253, 10) (2924, 9) (2924, 2)


In [4]:
# ID는 필요가 없다 왜?
train.drop('id', inplace=True, axis=1)
test.drop('id', inplace=True, axis=1)

In [5]:
train.head()

,Gender,Lenght,Diameter,Height,Whole Weight,Shucked Weight,Viscra Weight,Shell Weight,Target
0,M,0.605,0.470,0.115,1.1140,0.3925,0.2910,0.3100,15
1,I,0.430,0.315,0.095,0.3780,0.1750,0.0800,0.1045,8
2,I,0.580,0.490,0.195,1.3165,0.5305,0.2540,0.4100,18
3,M,0.535,0.405,0.175,1.2705,0.5480,0.3265,0.3370,13
4,I,0.310,0.235,0.090,0.1270,0.0480,0.0310,0.0400,6


In [6]:
test.head()

,Gender,Lenght,Diameter,Height,Whole Weight,Shucked Weight,Viscra Weight,Shell Weight
0,F,0.595,0.470,0.155,1.121,0.4515,0.1780,0.1550
1,M,0.580,0.450,0.150,0.927,0.2760,0.1815,0.3600
2,I,0.260,0.205,0.070,0.097,0.0415,0.0190,0.0305
3,M,0.590,0.460,0.130,1.102,0.4550,0.2055,0.3300
4,F,0.595,0.465,0.140,1.113,0.5175,0.2440,0.3050


In [7]:
train.shape

(1253, 9)

In [8]:
test.shape

(2924, 8)

In [9]:
submission.shape

(2924, 2)

In [10]:
#라벨인코딩을 하기 위함 dictionary map 생성 함수
def make_label_map(dataframe):
    label_maps = {}
    for col in dataframe.columns:
        if dataframe[col].dtype=='object':
            label_map = {'unknown':0}
            for i, key in enumerate(dataframe[col].unique()):
                label_map[key] = i+1  #새로 등장하는 유니크 값들에 대해 1부터 1씩 증가시켜 키값을 부여해줍니다.
            label_maps[col] = label_map
    print(label_maps)
    return label_maps

# 각 범주형 변수에 인코딩 값을 부여하는 함수
def label_encoder(dataframe, label_map):
    for col in dataframe.columns:
        if dataframe[col].dtype=='object':
            dataframe[col] = dataframe[col].map(label_map[col])
            dataframe[col] = dataframe[col].fillna(label_map[col]['unknown']) #혹시 모를 결측값은 unknown의 값(0)으로 채워줍니다.
    return dataframe

In [12]:
train_le = make_label_map(train[['Gender']])
gender_df = label_encoder(train[['Gender']], train_le)
train['Gender'] = gender_df[['Gender']]

{'Gender': {'unknown': 0, 'M': 1, 'I': 2, 'F': 3}}


/var/folders/hn/lsy2f4r16klfx3jt0tq69mh80000gn/T/ipykernel_54566/3387450210.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe[col] = dataframe[col].map(label_map[col])
/var/folders/hn/lsy2f4r16klfx3jt0tq69mh80000gn/T/ipykernel_54566/3387450210.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe[col] = dataframe[col].fillna(label_map[col]['unknown']) #혹시 모를 결측값은 unknown의 값(0)으로 채워줍니다.


In [13]:
print("2-M, 1-I, 0-F")

2-M, 1-I, 0-F


In [14]:
y_train

NameError: name 'y_train' is not defined

In [ ]:
x_train0 = train.iloc[:,0:8]

x_train = x_train0.loc[:,'Shell Weight':]
x_test = test.loc[:,'Shell Weight':]

from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
scaler.fit(x_train)
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.fit_transform(x_test)

print(x_train_scaled.shape, y_train.shape)
print(x_test_scaled.shape)

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout

def build_model(train_data, train_target):
    model = Sequential()
    model.add(Dense(128, activation='tanh', input_dim=train_data.shape[1]))
    model.add(Dropout(0.2))
    model.add(Dense(64, activation='tanh'))
    model.add(Dropout(0.2))
    model.add(Dense(32, activation='tanh'))
    model.add(Dense(train_target.shape[1], activation='softmax'))
    
    model.compile(optimizer='RMSProp', loss='categorical_crossentropy',
                 metrics=['acc', 'mae'])
    
    return model

model = build_model(x_train_scaled, y_train)
model.summary()

In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping

x_tr, x_val, y_tr, y_val = train_test_split(x_train_scaled, y_train, test_size=0.15,
                                           shuffle=True, random_state=SEED)

early_stoppping = EarlyStopping(monitor='val_loss', patience=10)
history = model.fit(x_tr, y_tr, batch_size=64, epochs=200,
                   validation_data=(x_val, y_val),
                   callbacks=[early_stoppping],
                   verbose=2)

In [ ]:
model.evaluate(x_val, y_val)

In [ ]:
y_pred_proba = model.predict(x_test)

In [ ]:
y_pred_label = np.argmax(y_pred_proba, axis=-1)
y_pred_label[:5]

In [ ]:
submission['Target']=y_pred_label.astype(int)
submission.head()

In [ ]:
submission.to_csv('submit.csv', index=False)